In [5]:
# autogen_example.py

import os
import random
import dotenv
from typing import List, Annotated
from pydantic import BaseModel, Field

import autogen
from autogen import AssistantAgent, UserProxyAgent
from autogen.cache import Cache
from nb_executor import NotebookExecutor

from protocol import SocialGraph, find_potential_peers

dotenv.load_dotenv()

# Mockup data
AGENTS = {
    "Agent1": {"links": ["Agent2", "Agent3", "Agent4"], "skills": ["Python", "Data Analysis"]},
    "Agent2": {"links": ["Agent1", "Agent3", "Agent5"], "skills": ["Java", "Machine Learning"]},
    "Agent3": {"links": ["Agent1", "Agent2", "Agent4"], "skills": ["Python", "NLP"]},
    "Agent4": {"links": ["Agent1", "Agent3", "Agent5"], "skills": ["C++", "Computer Vision"]},
    "Agent5": {"links": ["Agent2", "Agent4"], "skills": ["JavaScript", "Web Development"]},
}

# Initialize the social graph
social_graph = SocialGraph(AGENTS)

# LLM Configuration
llm_config = {
    "config_list": [{
    "openai_api_key": os.getenv("OPENAI_API_KEY"),
    "model": "gpt-4o-mini",
}],
    "timeout": 120,
}

# Define a dictionary to hold our agents
agents = {}

# Function to create agents
def create_agent(agent_id: str):
    system_message = f"You are {agent_id}. Your task is to find potential peers for a given task."
    chatbot = AssistantAgent(
        name=agent_id,
        system_message=system_message,
        llm_config=llm_config,
    )

    # Create a UserProxyAgent instance
    user_proxy = UserProxyAgent("user_proxy", 
                                code_execution_config={"work_dir": "./work", "use_docker": False},
                                is_termination_msg=lambda x: x.get("content", "") and (x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "") == ""),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,)


    # Register the find_potential_peers function
    @user_proxy.register_for_execution()
    @chatbot.register_for_llm(description="Find potential peers for a given task.")
    def find_peers(
        task_description: Annotated[str, "Description of the task"],
        agent_id: Annotated[str, "ID of the agent"],
        search_depth: Annotated[int, "Depth of search in the social graph"] = 2,
    ) -> List[str]:
        return find_potential_peers(
            task_description, agent_id, social_graph, search_depth
        )

    agents[agent_id] = {
        "chatbot": chatbot,
        "user_proxy": user_proxy,
    }

# Create agents for each agent in AGENTS
for agent_id in AGENTS.keys():
    create_agent(agent_id)

def propagate_request(initial_agent_id: str, task: str, max_hops: int = 2):
    visited = set()
    current_agents = [initial_agent_id]
    connections = []

    for hop in range(max_hops):
        next_agents = []
        for agent_id in current_agents:
            if agent_id in visited:
                continue
            visited.add(agent_id)

            agent = agents[agent_id]["chatbot"]
            user_proxy = agents[agent_id]["user_proxy"]

            # Start the conversation
            with Cache.disk() as cache:
                try:
                    res = user_proxy.initiate_chat(
                        agent,
                        message=f"Find potential peers for the task: '{task}'",
                        cache=cache,
                    )
                    result = res.chat_history[-1]["content"]
                except Exception as e:
                    print(f"Error running agent {agent_id}: {e}")
                    continue

            # Parse the result to get chosen peers
            chosen_peers = result.strip().split(', ')

            for peer in chosen_peers:
                if peer in visited:
                    continue
                if peer in AGENTS.keys():
                    path = social_graph.get_path(agent_id, peer)
                    if path:
                        connections.append((agent_id, peer, path))
                        if peer not in visited:
                            next_agents.append(peer)

        current_agents = next_agents

    return connections

# Run the simulation
task = "Develop a machine learning model for natural language processing"
initial_agent_id = "Agent1"
connections = propagate_request(initial_agent_id, task)

print(f"Task: {task}")
print(f"Initial Agent: {initial_agent_id}")
print("Connections formed:")
if connections:
    for connection in connections:
        print(f"{connection[0]} -> {connection[1]} (Path: {' -> '.join(connection[2])})")
else:
    print("No connections formed.")


user_proxy (to Agent1):

Find potential peers for the task: 'Develop a machine learning model for natural language processing'

--------------------------------------------------------------------------------
Error running agent Agent1: Completions.create() got an unexpected keyword argument 'openai_api_key'
Task: Develop a machine learning model for natural language processing
Initial Agent: Agent1
Connections formed:
No connections formed.
